In [5]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

#### Introducing data

In [13]:
# Projected travels into the future according to demographics and land use
P_is = np.array([750, 580, 480])    # produced
A_js = np.array([722, 786, 302])    # attracted

# Matrix of travels obtained with gravitational model, neutral calibration coeffs (simulating historical data)
gvalsr_m = np.array([[82, 140, 78], [43, 26, 31], [82, 31, 37]])

# Adjusted intial travels matrices
gvalsadjA = np.array([[81, 116, 103], [40, 20, 39], [78, 25, 47]])    # interative adjustment (Furness style)
gvalsadjB = np.array([[78, 112, 110], [40, 22, 38], [83, 26, 40]])    # weighted adjustment (new method)

# Calibration coefficients resulted accordingly
ccoeffsA = np.array([[0.49, 0.95, 1.46], [1.25, 1, 0.77], [1.41, 1.2, 0.21]])    # iterative ...
ccoeffsB = np.array([[0.51, 0.98, 1.36], [1.25, 0.91, 0.79], [1.33, 1.15, 0.25]])    # weighted ...

# Matrix of future travels obtained with gravitational model and ccoeffsA matrix
gvalsr_finA = np.array([[112, 377, 245], [298, 241, 49], [312, 168, 8]])
# Matrix of future travels obtained with gravitational model and ccoeffsB matrix
gvalsr_finB = np.array([[115, 392, 238], [302, 225, 54], [304, 169, 10]])

# Matrix of travels obtained with Furness method
travsc_furn = np.array([[99, 452, 199], [292, 194, 94], [320, 145, 16]])

# Matrix of travels obtained with Fratar method
travsc_frat = np.array([[105, 452, 192], [306, 186, 88], [330, 136, 14]])

# Matrix of travels obtained with Detroit method
travsc_detr = np.array([[103, 458, 199], [295, 191, 91], [318, 141, 15]])

# Matrix of travels obtained with weighted method (new method)
travsc_wgtd = np.array([[103, 487, 157], [295, 155, 131], [324, 144, 14]])

#### Flatten the original arrays

In [24]:
gvalsr_finAflat = gvalsr_finA.flatten()
gvalsr_finBflat = gvalsr_finB.flatten()

travsc_furnflat = travsc_furn.flatten()

travsc_fratflat = travsc_frat.flatten()

travsc_detrflat = travsc_detr.flatten()

travsc_wgtdflat = travsc_wgtd.flatten()

#### Create two dfs from flattened arrays

In [41]:
all_travs = pd.DataFrame(data=[gvalsr_finAflat, travsc_furnflat,
                               travsc_fratflat, travsc_detrflat,
                               travsc_wgtdflat]).T
all_travs.columns = ['gravit', 'furness', 'fratar', 'detroit', 'weighted']
all_travs.head()

,gravit,furness,fratar,detroit,weighted
0,112,99,105,103,103
1,377,452,452,458,487
2,245,199,192,199,157
3,298,292,306,295,295
4,241,194,186,191,155
